In [ ]:
# !pip install pytorch-pretrained-bert
# !pip3 install flashtext

     |████████████████████████████████| 133kB 2.8MB/s 
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9299 sha256=d6da6a2aae7d86b33b52fbe7fe2ab121845ec8e6a4b70d4b404acdecfe69d5e2
  Stored in directory: /root/.cache/pip/wheels/37/db/d7/fe74f7cb8e5c3afed90fe6f4967c933a6f13d81ab6b3d3128c
Successfully built flashtext


In [ ]:
# !git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git glue_data
# !python3 /content/glue_data/download_glue_data.py --data_dir glue_data --tasks=QQP

fatal: destination path 'glue_data' already exists and is not an empty directory.
	Completed!


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {
    width: 99% !important;
}   

div.cell.selected {
    border-left-width: 1px !important;
}</style>"))

SyntaxError: ignored

In [ ]:
# PREPROCESSING
import pandas as pd
import numpy as np
train_df = pd.read_csv('glue_data/QQP/train.tsv',delimiter="\t", error_bad_lines=False, index_col="id")
train_df.dropna(axis=0, inplace=True, how="any")
train_df.is_duplicate = train_df.is_duplicate.astype(int) #make 1s and 0s integers.
train_df = train_df.drop("qid1",1)
train_df = train_df.drop("qid2",1)
train_df.reset_index(drop=True, inplace=True)
print(train_df.shape)
print(train_df.columns)


test_df = pd.read_csv('glue_data/QQP/test.tsv', delimiter="\t",error_bad_lines=False, index_col="id")
test_df.dropna(axis=0, inplace=True, how="any")
test_df.reset_index(drop=True, inplace=True)
print(test_df.shape)
print(test_df.columns)

dev_df = pd.read_csv('glue_data/QQP/dev.tsv',delimiter="\t", error_bad_lines=False, index_col="id")
dev_df.dropna(axis=0, inplace=True, how="any")
dev_df.is_duplicate = dev_df.is_duplicate.astype(int) #make 1s and 0s integers.
dev_df = dev_df.drop("qid1",1)
dev_df = dev_df.drop("qid2",1)
dev_df.reset_index(drop=True, inplace=True)
print(dev_df.columns)
print(dev_df.shape)
print(dev_df.columns)

test_df["question1"][0]


b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(363177, 3)
Index(['question1', 'question2', 'is_duplicate'], dtype='object')
(390965, 2)
Index(['question1', 'question2'], dtype='object')
Index(['question1', 'question2', 'is_duplicate'], dtype='object')
(40371, 3)
Index(['question1', 'question2', 'is_duplicate'], dtype='object')


'Would the idea of Trump and Putin in bed together scare you, given the geopolitical implications?'

In [ ]:
# convert to lowercase
train_df["question1"] = train_df["question1"].str.lower() 
train_df["question2"] = train_df["question2"].str.lower() 
test_df["question1"] = test_df["question1"].str.lower() 
test_df["question2"] = test_df["question2"].str.lower() 
dev_df["question1"] = dev_df["question1"].str.lower() 
dev_df["question2"] = dev_df["question2"].str.lower() 

# expand contractions
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

from flashtext import KeywordProcessor

processor = KeywordProcessor(case_sensitive = False)
for k, v in contractions.items():
    processor.add_keyword(k,v)

for i, row in train_df.iterrows():
    train_df.at[i, 'question1'] = processor.replace_keywords(train_df["question1"][i])
    train_df.at[i, 'question2'] = processor.replace_keywords(train_df["question2"][i])

for i, row in test_df.iterrows():
    test_df.at[i, 'question1'] = processor.replace_keywords(test_df["question1"][i])
    test_df.at[i, 'question2'] = processor.replace_keywords(test_df["question2"][i])


for i, row in dev_df.iterrows():
    dev_df.at[i, 'question1'] = processor.replace_keywords(dev_df["question1"][i])
    dev_df.at[i, 'question2'] = processor.replace_keywords(dev_df["question2"][i])


In [ ]:
import re

# processing on test data?

# remove newline?

# --------remove punctations? No, just insert whitespaces before and after.-------------------
train_df["question1"] = train_df["question1"].replace(r'([^\w\s])', r' \1 ', regex=True)
train_df["question1"] = train_df["question1"].replace(r'\s{2,}', ' ', regex=True)
train_df["question2"] = train_df["question2"].replace(r'([^\w\s])', r' \1 ', regex=True)
train_df["question2"] = train_df["question2"].replace(r'\s{2,}', ' ', regex=True)

test_df["question1"] = test_df["question1"].replace(r'([^\w\s])', r' \1 ', regex=True)
test_df["question1"] = test_df["question1"].replace(r'\s{2,}', ' ', regex=True)
test_df["question2"] = test_df["question2"].replace(r'([^\w\s])', r' \1 ', regex=True)
test_df["question2"] = test_df["question2"].replace(r'\s{2,}', ' ', regex=True)

dev_df["question1"] = dev_df["question1"].replace(r'([^\w\s])', r' \1 ', regex=True)
dev_df["question1"] = dev_df["question1"].replace(r'\s{2,}', ' ', regex=True)
dev_df["question2"] = dev_df["question2"].replace(r'([^\w\s])', r' \1 ', regex=True)
dev_df["question2"] = dev_df["question2"].replace(r'\s{2,}', ' ', regex=True)

# --------remove non-ascii, already done!-----------------------------------------------------

# smaller than 512 -> feat size




how is the life of a math student ? could you describe your own experiences ? 
which level of prepration is enough for the exam jlpt5 ? 
0
how do i control my horny emotions ? 
how do you control your horniness ? 
1
what causes stool color to change to yellow ? 
what can cause stool to come out as little balls ? 
0
what can one do after mbbs ? 
what do i do after my mbbs ? 
1
where can i find a power outlet for my laptop at melbourne airport ? 
would a second airport in sydney , australia be needed if a high - speed rail link was created between melbourne and sydney ? 
0
how not to feel guilty since i am muslim and I am conscious we will not have sex together ? 
i do not beleive i am bulimic , but i force throw up atleast once a day after i eat something and feel guilty . should i tell somebody , and if so who ? 
0
how is air traffic controlled ? 
how do you become an air traffic controller ? 
0
what is the best self help book you have read ? why ? how did it change your life ? 
what a

In [ ]:
c = 0

for idx, row in train_df.iterrows():
    print(row[0])
    print(row[1])
    print(row[2])
    c+=1
    if(c==1000):
        break


how is the life of a math student ? could you describe your own experiences ? 
which level of prepration is enough for the exam jlpt5 ? 
0
how do i control my horny emotions ? 
how do you control your horniness ? 
1
what causes stool color to change to yellow ? 
what can cause stool to come out as little balls ? 
0
what can one do after mbbs ? 
what do i do after my mbbs ? 
1
where can i find a power outlet for my laptop at melbourne airport ? 
would a second airport in sydney , australia be needed if a high - speed rail link was created between melbourne and sydney ? 
0
how not to feel guilty since i am muslim and I am conscious we will not have sex together ? 
i do not beleive i am bulimic , but i force throw up atleast once a day after i eat something and feel guilty . should i tell somebody , and if so who ? 
0
how is air traffic controlled ? 
how do you become an air traffic controller ? 
0
what is the best self help book you have read ? why ? how did it change your life ? 
what a

In [ ]:
train2_df = pd.read_csv('glue_data/QQP/train.tsv',delimiter="\t", error_bad_lines=False, index_col="id")

c = 0
for idx, row in train2_df.iterrows():
    print(row[2])
    print(row[3])
    print(row[4])
    c+=1
    if(c==1000):
        break


b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


How is the life of a math student? Could you describe your own experiences?
Which level of prepration is enough for the exam jlpt5?
0.0
How do I control my horny emotions?
How do you control your horniness?
1.0
What causes stool color to change to yellow?
What can cause stool to come out as little balls?
0.0
What can one do after MBBS?
What do i do after my MBBS ?
1.0
Where can I find a power outlet for my laptop at Melbourne Airport?
Would a second airport in Sydney, Australia be needed if a high-speed rail link was created between Melbourne and Sydney?
0.0
How not to feel guilty since I am Muslim and I'm conscious we won't have sex together?
I don't beleive I am bulimic, but I force throw up atleast once a day after I eat something and feel guilty. Should I tell somebody, and if so who?
0.0
How is air traffic controlled?
How do you become an air traffic controller?
0.0
What is the best self help book you have read? Why? How did it change your life?
What are the top self help books I 

In [ ]:
# train_df.to_csv('train.tsv', sep='\t', index=False, header=False)
# test_df.to_csv('test.tsv', sep='\t', index=False, header=False)
# dev_df.to_csv('dev.tsv', sep='\t', index=False, header=False)
